In [1]:
import os
import pickle
import csv
import numpy as np

from sklearn.linear_model import OrthogonalMatchingPursuit, OrthogonalMatchingPursuitCV
from sklearn.pipeline import make_pipeline

#### Функция чтения набора данных

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='H_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for pr in pr_list:
                pr_arr_row.append(row[pr])

            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float64)
    y = np.asarray(y_arr, dtype=np.float64)
    return X, y

#### Функция формирования тестового набора данных с подстановкой нормированных значений

In [3]:
def test_norm(x, pr_list, norms):
    x_norm = np.copy(x)
    for col, pr in enumerate(pr_list):
        if pr in norms:
            x_norm[:, col:col+1] = norms[pr]
    return x_norm

In [4]:
def write_dataset_csv(year, dataset, dataset_name, fieldnames, pr_group, mode='training'):
    if mode == 'estimation':
        dir_path = f'results/Estimation/{year}/{dataset_name}/group-{pr_group}/'
        file_name = f'{dataset_name}-гр{pr_group}-Оценка.csv'
    elif mode == 'training':
        dir_path = f'results/Models/{year}/'
        file_name = f'{dataset_name}-гр{pr_group}-Обучение.csv'
    elif mode == 'forecast':
        dir_path = f'results/Forecast/{year}/'
        file_name = f'{dataset_name}-гр{pr_group}-Прогноз.csv'
    else:
        ...
    
    with open(
        f'{dir_path}'
        f'{file_name}', 
        'w', newline='', encoding='utf-8'
    ) as csvfile:
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, 
                                delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


### Функция прогнозирования

In [5]:
def forecast(year, norms=False):

    fieldnames = [
        'Predictors', 
        'Equations', 
        'Method', 
        'Criterion', 
        'Correlation', 
        'Pm',
        'R2',        
        'R2_t',
        'Prediction',
    ]
    
    predict_data_dir = f'data/{year}/Predict'
    
    models_dir = f'results/Models/{year}'

    # Получить список файлов .ipnb из results\Models\<year>
    file_list = tuple(filter(lambda fn: '.pickle' in fn, os.listdir(models_dir)))
    # print(file_list)

    # Сделать множество из кортежей (Название-датасета, группа), преобразовать в список, отсортировать
    dataset_group = set()
    for fn in file_list:
        dataset, year, group, model = fn.split('_')
        dataset_group |= {(dataset, group.split('гр')[1])}
    ds_list = sorted(dataset_group)
    # print(ds_list)

    # Для каждого элемента списка создать result_list[one_model_raw: dict]
    
    for ds, group in ds_list:
        result_list = []
        ds_models = filter(lambda fn: ds in fn and group in fn, file_list)
        for file_name in ds_models:
            with open(f'results/Models/{year}/{file_name}', 'rb') as f:
                model_info = pickle.load(f)
                model = model_info['Model_full']
                #model = model_info['Model_train']
                
                # Прочитать навые признаки (предикторы) для прогноза
                X_new, y = get_river_dataset(f'{predict_data_dir}/{ds}.csv', pr_list=model_info['Predictors_list'])

                # Подстановка нормированных значений для целей тестирования
                if norms:
                    # Подстановка норм в исходный набор признаков
                    X_new = test_norm(X_new, model_info['Predictors_list'], model_info['Norms_data'])

                print(model_info['Dataset_name'])
                print(model_info['Predictors_list'])
                print(model_info['Norms_data'])
                print(model_info['Method'])
                print("y")
                print(y)
                print(f'R2={model_info["R2"]}, R2_t={model_info["R2_t"]}')
                
                y_predicted = np.ravel(model.predict(X_new))
                print("y_predicted")
                print(y_predicted)
                print('X_new')
                print(X_new)
                print(model)
                model_info['Prediction'] = y_predicted[-1]
                result_list.append(model_info)
                # Сортировка результатов по каждому датасету
                result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))
                
                # Запись в .csv файл
                write_dataset_csv(year, result_list, ds, fieldnames, pr_group=group, mode='forecast')
                print('------------------------------------------------------------------------------')
    
    

In [6]:
forecast(2024, norms=False)

Вилия-Стешицы
['S_2802', 'Smax', 'H_2802', 'X', 'X1', 'X2', 'X3', 'Xs', 'L_max', 'L_2802', 'Q12', 'Q01', 'Q02', 'Y_sum']
None
OMP7
y
[281. 292. 275. 291. 168. 195. 166. 198. 286. 277. 207. 292. 216. 198.
 296. 220. 279. 236. 227. 233. 321. 242. 235. 281. 200. 267. 266. 200.
 212. 281.]
R2=0.8488614707129319, R2_t=0.8235339018679704
y_predicted
[300.68587497 281.40184865 287.93943673 264.08207211 176.62641453
 230.9573563  161.5907951  201.98826667 286.15784913 287.91857213
 198.16711425 300.5358806  210.39751134 210.66075721 294.75892495
 253.81390639 271.86292352 227.70470309 238.01258577 229.34033746
 281.49236724 263.20793023 244.24610926 277.72031805 210.39988732
 252.06000311 252.05325105 192.22130295 192.32472806 268.68137472]
X_new
[[ 93.    99.   139.   155.9   38.5   48.   184.2  146.1   83.    78.
    6.2    4.66   4.5   32.5 ]
 [ 94.    96.   143.   139.1   16.7   14.1  244.2  142.2   88.    78.
    7.29   4.4    5.77  36.8 ]
 [105.   105.   140.   102.5   30.4   30.4  155.9